# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

GOOGLE_API_KEY=****re4I
LANGSMITH_TRACING=true
LANGSMITH_API_KEY=****ad5c
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [7]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (f75be5e8-6d67-4a9f-97ee-b833fdb52460)
 Call ID: f75be5e8-6d67-4a9f-97ee-b833fdb52460
  Args:
    query: SELECT CustomerId, FirstName, LastName FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT CustomerId, FirstName, LastName FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (e6ddae7f-dc6e-46b4-ba42-aeeba6820c74)
 Call ID: e6ddae7f-dc6e-46b4-ba42-aeeba6820c74
  Args:
    query

In [8]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (f0c7dd80-06a0-4fad-a979-f09f6e04e153)
 Call ID: f0c7dd80-06a0-4fad-a979-f09f6e04e153
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (7a88fc82-b166-494f-8e9c-8517a0a7eae8)
 Call ID: 7a88fc82-b166-494f-8e9c-8517a0a7eae8
  Args:
    query: SELECT Title FROM Album LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('For Those About To Roc

## Add memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

In [10]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="google_genai:gemini-2.5-flash",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [11]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (1bc90c3d-580b-4c90-8d12-12f098d64665)
 Call ID: 1bc90c3d-580b-4c90-8d12-12f098d64665
  Args:
    query: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT CustomerId FROM customers WHERE FirstName = 'Frank' AND LastName = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (ed10a157-4be2-4b31-ad62-1f28192cafdb)
 Call ID: ed10a157-4be2-4b31-ad62-1f28192cafdb
  Args:
    query: SELECT name FROM sqlite_master WHERE typ

In [12]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (cb3af5e3-ada8-46e5-b4f4-88faf8a9d07c)
 Call ID: cb3af5e3-ada8-46e5-b4f4-88faf8a9d07c
  Args:
    query: SELECT InvoiceId FROM Invoice WHERE CustomerId = 16 ORDER BY InvoiceDate DESC LIMIT 1
================================= Tool Message =================================
Name: execute_sql

[(374,)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (7ed61e52-f4d1-420c-8f2d-86e3c635ac0e)
 Call ID: 7ed61e52-f4d1-420c-8f2d-86e3c635ac0e
  Args:
    query: SELECT T.Name FROM Track AS T JOIN InvoiceLine AS IL ON T.TrackId = IL.TrackId WHERE IL.InvoiceId = 374
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',),

## Try your own queries
Now that there is memory, check the agents recall!

In [13]:
question = "What is the total number of customer?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "2"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the total number of customer?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (da0f7f4b-3844-4bd6-9714-4664f099d4c7)
 Call ID: da0f7f4b-3844-4bd6-9714-4664f099d4c7
  Args:
    query: SELECT COUNT(*) FROM customers;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT COUNT(*) FROM customers;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (aa66b7d8-0ad9-4a52-8a9f-c68bb4b8f57b)
 Call ID: aa66b7d8-0ad9-4a52-8a9f-c68bb4b8f57b
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), (

In [14]:
question = "What is the total number after 12 of them exited?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "2"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the total number after 12 of them exited?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'The total number of customers after 12 of them exited would be 59 - 12 = 47.', 'extras': {'signature': 'CugBAXLI2nzw5Bt39hdZQk2P1CuP2EHbXKmpyx5w7wAEwE2wPmAIdpGWHRioXhTJbkiVuyHSq5STkFX04Lokg1ZUaMlP9A5lG/TVEgLxXaCutd5MBVTKiTKcRXHZWBBszW4KsoD/4gxfQtksnhWOY5kggx5ggJtL11vivvckxnhwgWetHepCkZaQLgtiIoEiRnh+lMsxtvsLBhbFXBI2fXlYMwHFQIc8qlIMWw3KwhnsE5/S5YE8JkEldmvqDf6j3zMoIfdI1rLpRL4N1rU5pCeTQ6q6cvfuEZ9/mravyuhu97SmzeArdbObfA=='}}]
